 !wget --no-check-certificate \
     https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
     -O /tmp/bbc-text.csv
     
90%

In [62]:
import csv
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalAveragePooling1D, Dense, BatchNormalization 
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import Callback

In [42]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8

In [82]:
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        acc=0.95
        if(logs.get('val_accuracy')>acc):
            print("\nReached {acc} accuracy so cancelling training!")
            self.model.stop_training = True

In [43]:

stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
print(len(stopwords))
# Expected Output
# 153

153


In [5]:
labels=[]
sentences=[]
with open('/tmp/bbc_text/bbc-text.csv') as f:
    reader=csv.reader(f)
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence=row[1]
        for stopword in stopwords:
            token=f" {stopword} "
            sentence=sentence.replace(token, " ")
        sentences.append(sentence)
        

In [24]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

In [25]:
tokenizer.fit_on_texts(sentences)

In [26]:
n_words=len(tokenizer.word_index)
n_words

29714

In [31]:
seq=tokenizer.texts_to_sequences(sentences)

In [33]:
padded=pad_sequences(seq, maxlen=max_length, padding='pre')

In [44]:
padded=np.array(padded)

In [36]:
label_dict={}
for label in set(labels):
    label_dict[label]=len(label_dict)

In [45]:
int_labels = np.array(list(map(label_dict.get, labels)))

In [65]:
n=len(int_labels)
split_at=int(n*training_portion)
X_train=padded[:split_at]
y_train=int_labels[:split_at]
X_valid=padded[split_at:]
y_valid=int_labels[split_at:]

In [145]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    GlobalAveragePooling1D(),
    #Dense(16, activation='relu'),
    BatchNormalization(),
    Dense(5, activation='softmax')
])

In [146]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 120, 16)           16000     
_________________________________________________________________
global_average_pooling1d_20  (None, 16)                0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 16)                64        
_________________________________________________________________
dense_39 (Dense)             (None, 5)                 85        
Total params: 16,149
Trainable params: 16,117
Non-trainable params: 32
_________________________________________________________________


In [147]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(clipvalue=1.0), metrics=['accuracy'])

In [148]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[myCallback()])

Epoch 1/100
56/56 [==============================] - 0s 6ms/step - loss: 1.3343 - accuracy: 0.6736 - val_loss: 1.5773 - val_accuracy: 0.4719
Epoch 2/100
56/56 [==============================] - 0s 5ms/step - loss: 0.8102 - accuracy: 0.8899 - val_loss: 1.5184 - val_accuracy: 0.4629
Epoch 3/100
56/56 [==============================] - 0s 5ms/step - loss: 0.4858 - accuracy: 0.9416 - val_loss: 1.4393 - val_accuracy: 0.5146
Epoch 4/100
56/56 [==============================] - 0s 4ms/step - loss: 0.3200 - accuracy: 0.9601 - val_loss: 1.3358 - val_accuracy: 0.6180
Epoch 5/100
56/56 [==============================] - 0s 4ms/step - loss: 0.2269 - accuracy: 0.9691 - val_loss: 1.1994 - val_accuracy: 0.8067
Epoch 6/100
56/56 [==============================] - 0s 4ms/step - loss: 0.1687 - accuracy: 0.9781 - val_loss: 1.0341 - val_accuracy: 0.8809
Epoch 7/100
56/56 [==============================] - 0s 4ms/step - loss: 0.1369 - accuracy: 0.9809 - val_loss: 0.8427 - val_accuracy: 0.9011
Epoch 8/100
5

In [149]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
56/56 [==============================] - 0s 5ms/step - loss: 0.0251 - accuracy: 0.9994 - val_loss: 0.1832 - val_accuracy: 0.9483
Epoch 2/100
56/56 [==============================] - 0s 4ms/step - loss: 0.0223 - accuracy: 1.0000 - val_loss: 0.1790 - val_accuracy: 0.9461
Epoch 3/100
56/56 [==============================] - 0s 4ms/step - loss: 0.0187 - accuracy: 0.9994 - val_loss: 0.1770 - val_accuracy: 0.9483
Epoch 4/100
56/56 [==============================] - 0s 5ms/step - loss: 0.0158 - accuracy: 1.0000 - val_loss: 0.1814 - val_accuracy: 0.9461
Epoch 5/100
56/56 [==============================] - 0s 5ms/step - loss: 0.0131 - accuracy: 1.0000 - val_loss: 0.1799 - val_accuracy: 0.9461
Epoch 6/100
56/56 [==============================] - 0s 5ms/step - loss: 0.0140 - accuracy: 1.0000 - val_loss: 0.1845 - val_accuracy: 0.9461
Epoch 7/100
56/56 [==============================] - ETA: 0s - loss: 0.0120 - accuracy: 1.00 - 0s 4ms/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 0

56/56 [==============================] - 0s 5ms/step - loss: 7.6525e-04 - accuracy: 1.0000 - val_loss: 0.2497 - val_accuracy: 0.9371
Epoch 59/100
56/56 [==============================] - 0s 5ms/step - loss: 5.7945e-04 - accuracy: 1.0000 - val_loss: 0.2451 - val_accuracy: 0.9416
Epoch 60/100
56/56 [==============================] - 0s 5ms/step - loss: 7.2836e-04 - accuracy: 1.0000 - val_loss: 0.2425 - val_accuracy: 0.9393
Epoch 61/100
56/56 [==============================] - 0s 5ms/step - loss: 8.3374e-04 - accuracy: 1.0000 - val_loss: 0.2937 - val_accuracy: 0.9371
Epoch 62/100
56/56 [==============================] - 0s 5ms/step - loss: 6.8750e-04 - accuracy: 1.0000 - val_loss: 0.2603 - val_accuracy: 0.9416
Epoch 63/100
56/56 [==============================] - 0s 4ms/step - loss: 5.9395e-04 - accuracy: 1.0000 - val_loss: 0.2571 - val_accuracy: 0.9393
Epoch 64/100
56/56 [==============================] - 0s 5ms/step - loss: 6.8581e-04 - accuracy: 1.0000 - val_loss: 0.2482 - val_accuracy